In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         pass
#         print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import argparse
import os
import random
import shutil
import time
import warnings
from enum import Enum

import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Subset


In [3]:
# !pip install torchinfo torchmetrics

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.datasets as datasets
from torchvision import transforms
import random
from torchmetrics import CharErrorRate
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import pandas as pd
from torchinfo import summary

In [5]:
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnext50_32x4d, ResNeXt50_32X4D_Weights

In [6]:
import numpy as np
from glob import glob
from matplotlib.pyplot import imread
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil
import time
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy

In [7]:
#shutil.rmtree(imfolder_path)
#shutil.rmtree(os.path.join(r'/kaggle/working', 'test'))
try:
    os.mkdir(os.path.join(r'/kaggle/working', 'train'))
except FileExistsError:
    pass
try:
    os.mkdir(os.path.join(r'/kaggle/working', 'test'))
except FileExistsError:
    pass

mode = r'train'
path = os.path.join(r'/kaggle/input/vk-made-sports-image-classification/', mode)
imfolder_path = os.path.join(r'/kaggle/working', mode)
data_labels = pd.read_csv(path + '.csv')
n_samples = len(data_labels)

wrong_channel_count = []
for idx, row in tqdm(data_labels.iloc[:n_samples].iterrows()):
    item_path = os.path.join(path, row.image_id)
    item_imfolder_dir =  os.path.join(imfolder_path, row.label)
    item_imfolder_dir_test =  os.path.join(r'/kaggle/working/test', row.label)
#     print(item_imfolder_dir, item_imfolder_dir_test)
#     input()
    if not os.path.exists(item_imfolder_dir):
        os.mkdir(item_imfolder_dir)
    if not os.path.exists(item_imfolder_dir_test):
        os.mkdir(item_imfolder_dir_test)

    if not os.path.exists(item_path):
        print(item_path)
        continue
#     with Image.open(item_path).convert('RGB') as img:
#         chn_cnt = np.asarray(img.getdata()).shape[1]
        
#         if chn_cnt != 3:
#             print('here again')
#             wrong_channel_count.append(chn_cnt)
#             continue
    try:
        if np.random.random() < 0.85:
            os.symlink(item_path, os.path.join(item_imfolder_dir, row.image_id))
        else:
            os.symlink(item_path, os.path.join(item_imfolder_dir_test, row.image_id))
#         if not os.path.exists(os.path.join(item_imfolder_dir, row.image_id)):
#             print(item_path, 'exists ', os.path.exists(item_path))
#             print(os.path.join(item_imfolder_dir, row.image_id))
#             input()
        #print(os.path.join(item_imfolder_path, row.image_id))
    except FileExistsError:
        pass

45373it [01:30, 502.30it/s]


In [8]:
len(os.listdir(r'/kaggle/working/train/swimming'))

1071

In [9]:


random_transforms = transforms.RandomApply(nn.ModuleList([
    transforms.RandomAffine(degrees=(-2, 2)), 
    transforms.RandomAdjustSharpness(1.2),
    transforms.RandomPerspective(distortion_scale=0.1), 
    transforms.RandomHorizontalFlip()
]))



Based on:

## https://github.com/pytorch/examples/blob/main/imagenet/main.py
## https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [10]:
MODEL_CLASS = resnext50_32x4d
MODEL_WEIGHTS = ResNeXt50_32X4D_Weights.IMAGENET1K_V2
MODEL_NAME = 'resnext50_32x4d'

In [11]:
MODEL_CLASS = resnet50
MODEL_WEIGHTS = ResNet50_Weights.DEFAULT
MODEL_NAME = 'resnet50'

In [12]:
best_acc1 = 0
def main_worker(resume=False):
    global best_acc1

    device = torch.device("cuda")

    model = MODEL_CLASS(weights=MODEL_WEIGHTS)  
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 30)    
    model = torch.nn.DataParallel(model).cuda()
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), 1e-3,
                                momentum=0.9,
                                weight_decay=1e-4)
    scheduler = StepLR(optimizer, step_size=20, gamma=0.2)
    start_epoch = 0
    if resume:
        checkpoint = torch.load(f'/kaggle/working/{MODEL_NAME}_chckpt.pth.tar', map_location='cuda')
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        start_epoch = checkpoint['epoch']
        best_acc1 = checkpoint['best_acc1']
    
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

    train_dataset = datasets.ImageFolder(
        r'/kaggle/working/train',
        transforms.Compose([
            transforms.RandomResizedCrop(224),
            random_transforms,
            transforms.ToTensor(),
            transforms.RandomErasing(p=0.6, scale=(0.02, 0.2)),
            normalize
        ]))

    val_dataset = datasets.ImageFolder(
        r'/kaggle/working/test',
        transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize
        ]))

    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=64, shuffle=True,
        pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=64, shuffle=False,
        pin_memory=True)

    for epoch in range(start_epoch, start_epoch + 10):
        train_epoch(train_loader, model, criterion, optimizer, epoch, device)
        
        acc1 = validate(val_loader, model, criterion)
        
        scheduler.step()
        
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)
        print('---------------------------------')
        print(f"val acc: {100 * acc1:6.2f}")
    
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': 'cuda',
            'state_dict': model.state_dict(),
            'best_acc1': best_acc1,
            'optimizer' : optimizer.state_dict(),
            'scheduler' : scheduler.state_dict()
        }, is_best)
    


In [13]:
def train_epoch(train_loader, model, criterion, optimizer, epoch, device):
    
    model.train()

    idx_progress = 0
    
    acc1_sum = 0
    val_size = 0
    for i, (images, target) in enumerate(train_loader):
        images = images.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        output = model(images)
        loss = criterion(output, target)

        _, pred = torch.topk(output, 1)
        correct = pred.squeeze() == target
        acc1_sum += correct.sum()
        val_size += images.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        idx_progress += images.size(0)
        if idx_progress > 4000:
            idx_progress -= 4000
            print(f"train acc1: {100 * acc1_sum/val_size:6.2f}")


def validate(val_loader, model, criterion):        

    model.eval()
    acc1_sum = 0
    val_size = 0
    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.cuda(non_blocking=True)

            target = target.cuda(non_blocking=True)

            output = model(images)
            loss = criterion(output, target)

            _, pred = torch.topk(output, 1)
            correct = pred.squeeze() == target
            acc1_sum += correct.sum()
            val_size += images.size(0)

    return acc1_sum / val_size


In [14]:

def save_checkpoint(state, is_best, filename=f'/kaggle/working/{MODEL_NAME}_chckpt.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, f'/kaggle/working/{MODEL_NAME}_best.pth.tar')

In [15]:
#main_worker(resume=True)


In [16]:
device = 'cuda'
model = MODEL_CLASS(weights=MODEL_WEIGHTS)  
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 30)
model = nn.DataParallel(model).cuda()
criterion = nn.CrossEntropyLoss().to(device)

optimizer = torch.optim.SGD(model.parameters(), 1e-3,
                            momentum=0.9,
                            weight_decay=1e-4)
    
    
scheduler = StepLR(optimizer, step_size=20, gamma=0.2)
checkpoint = torch.load(f'/kaggle/working/{MODEL_NAME}_best.pth.tar', map_location='cuda')
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
scheduler.load_state_dict(checkpoint['scheduler'])
start_epoch = checkpoint['epoch']
best_acc1 = checkpoint['best_acc1']

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/resnet50_best.pth.tar'

In [ ]:
model.eval()

data4test = pd.read_csv(r'/kaggle/input/vk-made-sports-image-classification/test.csv')

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

batch = torch.empty(64, 3, 224, 224, dtype=torch.float)
label_idx = np.empty((0, 1), dtype=np.int)

for idx, row in tqdm(data4test.iterrows()):
    item_path = os.path.join(r'/kaggle/input/vk-made-sports-image-classification/test', row.image_id)
    with Image.open(item_path).convert("RGB") as img:
        batch[idx % 64] = preprocess(img) 
        
    if (idx + 1) % 64 == 0:
        output = model(batch)
        
        _, pred = output.topk(1, 1, True, True) 
        label_idx = np.vstack((label_idx, pred.cpu().numpy()))

output = model(batch)
        
_, pred = output.topk(1, 1, True, True) 
label_idx = np.vstack((label_idx, pred.cpu().numpy()))

In [ ]:
classes = sorted(['sambo', 'handball', 'ski_race', 'boxing', 'skating', 'javelin-throwing', 'swimming', 'basketball', 'alpinism', 'water_polo', 'football', 'badminton', 'running', 'rugby', 'greco-Roman_wrestling', 'gymnastics', 'pole_vault', 'golf', 'surfing', 'tennis', 'boating', 'sailing', 'arm_wrestling', 'velo', 'horseback_riding', 'taekwondo', 'fencing', 'hockey', 'volleyball', 'archery'])
len(classes)

In [ ]:
idx2lbl = np.vectorize(lambda x: classes[x])
data4test['label'] = idx2lbl(label_idx[:len(data4test)])
data4test.to_csv('/kaggle/working/submission.csv', index=False)
data4test.to_csv('submission.csv', index=False)